In [3]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('vgg16_xray_classification_model.h5')

# Define image dimensions
img_width, img_height = 224, 224

class XRayClassifierApp:
    def __init__(self, root):
        self.root = root
        self.root.title("X-Ray Image Classifier")

        self.label = tk.Label(root, text="Select an X-ray image:")
        self.label.pack(pady=10)

        self.browse_button = tk.Button(root, text="Browse", command=self.browse_image)
        self.browse_button.pack(pady=10)

        self.predict_button = tk.Button(root, text="Predict", command=self.predict_image)
        self.predict_button.pack(pady=10)

        self.result_label = tk.Label(root, text="")
        self.result_label.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(title="Select an X-ray image", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.display_image(file_path)

    def display_image(self, file_path):
        img = Image.open(file_path)
        img = img.resize((img_width, img_height), Image.BICUBIC)
        img = ImageTk.PhotoImage(img)

        if hasattr(self, 'image_label'):
            self.image_label.pack_forget()

        self.image_label = tk.Label(root, image=img)
        self.image_label.image = img
        self.image_label.pack(pady=10)

        self.image_path = file_path

    def preprocess_image(self, image_path):
        img = Image.open(image_path)
        img = img.resize((img_width, img_height), Image.BICUBIC)

        # Convert to grayscale
        img = img.convert('L')

        # Replicate single channel to create three channels
        img_array = np.array(img)
        img_array = np.stack((img_array,) * 3, axis=-1)

        img_array = img_array / 255.0
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array

    def predict_image(self):
        if hasattr(self, 'image_path'):
            preprocessed_img = self.preprocess_image(self.image_path)

            try:
                prediction = model.predict(preprocessed_img)

                # Assuming class indices: 0 - "COVID-19", 1 - "Lung Opacity", 2 - "NORMAL", 3 - "PNEUMONIA", 4 - "Tuberculosis"
                class_labels = ["COVID-19", "Lung Opacity", "NORMAL", "PNEUMONIA", "Tuberculosis"]
                predicted_class_index = np.argmax(prediction)
                predicted_class = class_labels[predicted_class_index]

                result_text = f"Prediction: {predicted_class}"
                self.result_label.config(text=result_text)
            except Exception as e:
                print(f"Error predicting image: {e}")
                self.result_label.config(text="Error predicting image.")
        else:
            self.result_label.config(text="Please select an image first.")

if __name__ == "__main__":
    root = tk.Tk()
    app = XRayClassifierApp(root)
    root.mainloop()


1/1 [==============================] - 0s 220ms/step
